In [1]:
## THIS CELL SHOULD BE IN ALL VSCODE NOTEBOOKS ##

MARKET = "NSE"

# Set the root
from from_root import from_root
ROOT = from_root()

import pandas as pd
from loguru import logger

pd.options.display.max_columns = None
pd.set_option('display.precision', 2)

from pathlib import Path
import sys

# Add `src` and ROOT to _src.pth in .venv to allow imports in VS Code
from sysconfig import get_path

if "src" not in Path.cwd().parts:
    src_path = str(Path(get_path("purelib")) / "_src.pth")
    with open(src_path, "w") as f:
        f.write(str(ROOT / "src\n"))
        f.write(str(ROOT))
        if str(ROOT) not in sys.path:
            sys.path.insert(1, str(ROOT))

# Start the Jupyter loop
from ib_async import util

util.startLoop()

logger.add(sink=ROOT / "log" / "ztest.log", mode="w")

1

# AI generated SNP test

In [ ]:
import yfinance as yf
import numpy as np
from scipy.stats import norm
import pandas as pd

# Fetch the option data for 'INTC'
intc = yf.Ticker("INTC")
option_chain = intc.option_chain()

# Extract the relevant data from the option chain
strikes = option_chain.calls.strike.unique()
expiration_dates = option_chain.calls.expiration.unique()

# Define the function to calculate implied volatility
def implied_volatility(S, K, T, r, option_price, option_type):
    """
    Calculate the implied volatility of an option using the Black-Scholes formula.
    
    Parameters:
    S (float): Current stock price
    K (float): Strike price
    T (float): Time to expiration in years
    r (float): Risk-free interest rate
    option_price (float): Current option price
    option_type (str): 'call' or 'put'
    
    Returns:
    float: Implied volatility
    """
    d1 = (np.log(S / K) + (r + 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    
    if option_type == 'call':
        model_price = S * norm.cdf(d1) - K * np.exp(-r * T) * norm.cdf(d2)
    elif option_type == 'put':
        model_price = K * np.exp(-r * T) * norm.cdf(-d2) - S * norm.cdf(-d1)
    
    return sigma

# Create a list to store the data
data = []

# Calculate the implied volatility and option price for each strike and expiration date
for expiration in expiration_dates:
    for strike in strikes:
        call_data = option_chain.calls[(option_chain.calls.expiration == expiration) & (option_chain.calls.strike == strike)]
        put_data = option_chain.puts[(option_chain.puts.expiration == expiration) & (option_chain.puts.strike == strike)]
        
        if not call_data.empty:
            call_price = call_data.lastPrice.values[0]
            call_iv = implied_volatility(intc.price, strike, (expiration - intc.current_date).days / 365, 0.01, call_price, 'call')
            data.append({
                'Symbol': 'INTC',
                'Expiry': expiration,
                'Days to Expiry': (expiration - intc.current_date).days / 365,
                'Strike': strike,
                'Type': 'Call',
                'Price': call_price,
                'IV': call_iv
            })
        
        if not put_data.empty:
            put_price = put_data.lastPrice.values[0]
            put_iv = implied_volatility(intc.price, strike, (expiration - intc.current_date).days / 365, 0.01, put_price, 'put')
            data.append({
                'Symbol': 'INTC',
                'Expiry': expiration,
                'Days to Expiry': (expiration - intc.current_date).days / 365,
                'Strike': strike,
                'Type': 'Put',
                'Price': put_price,
                'IV': put_iv
            })

# Create the DataFrame
df = pd.DataFrame(data)